In [25]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision import transforms
from torchvision.transforms import ToTensor
from torchvision.transforms import v2
from torchvision.io import read_image
import numpy as np
from torch.utils.data import Subset

In [26]:
preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [27]:
batch_size = 32

In [28]:
training_data = datasets.CIFAR100(
    root="data",
    train=True,
    transform=preprocess,
    download=True
)

test_data = datasets.CIFAR100(
    root="data",
    train=False,
    transform=preprocess,
    download=True
)

train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

Files already downloaded and verified
Files already downloaded and verified


In [29]:
import torchvision.models

In [30]:
model = torchvision.models.resnet18()

In [31]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')


print(device)

cuda:0


In [32]:
model.to(device)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [33]:
initial_subset_size = 3125  # Starting with N samples
increase_factor = 2  # Increase the subset size by this factor
epochs = 25  # Total number of epochs
increase_interval = 5  # Number of epochs before increasing the subset size
learning_rate = 1e-3
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [34]:
from sklearn.model_selection import train_test_split

def create_stratified_subset_dataloader(dataset, subset_size):
    targets = dataset.targets
    indices = np.arange(len(targets))
    subset_indices, _ = train_test_split(indices, train_size=subset_size, stratify=targets)
    subset = Subset(dataset, subset_indices)
    return DataLoader(subset, batch_size=batch_size, shuffle=True)



In [35]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    # Set the model to training mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        X = X.to(device)
        y = y.to(device)
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * batch_size + len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test_loop(dataloader, model, loss_fn):
    # Set the model to evaluation mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    model.eval()
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    # Evaluating the model with torch.no_grad() ensures that no gradients are computed during test mode
    # also serves to reduce unnecessary gradient computations and memory usage for tensors with requires_grad=True
    with torch.no_grad():
        for X, y in dataloader:
            X = X.to(device)
            y = y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [36]:
current_subset_size = initial_subset_size
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_dataloader = create_stratified_subset_dataloader(training_data, current_subset_size) 
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)

    # Increase the subset size at specified intervals
    if (t + 1) % increase_interval == 0 and current_subset_size * increase_factor <= len(training_data):
        current_subset_size *= increase_factor
        if current_subset_size == 50000: current_subset_size = 25000

print("Done!")

Epoch 1
-------------------------------
loss: 6.977133  [   32/ 3125]
Test Error: 
 Accuracy: 4.9%, Avg loss: 4.453303 

Epoch 2
-------------------------------
loss: 4.422045  [   32/ 3125]
Test Error: 
 Accuracy: 5.2%, Avg loss: 4.388811 

Epoch 3
-------------------------------
loss: 4.309286  [   32/ 3125]
Test Error: 
 Accuracy: 6.5%, Avg loss: 4.089954 

Epoch 4
-------------------------------
loss: 3.956225  [   32/ 3125]
Test Error: 
 Accuracy: 6.8%, Avg loss: 4.025901 

Epoch 5
-------------------------------
loss: 4.067620  [   32/ 3125]
Test Error: 
 Accuracy: 9.4%, Avg loss: 3.952619 

Epoch 6
-------------------------------
loss: 4.077469  [   32/ 6250]
loss: 3.993210  [ 3232/ 6250]
Test Error: 
 Accuracy: 9.8%, Avg loss: 3.921487 

Epoch 7
-------------------------------
loss: 3.687742  [   32/ 6250]
loss: 3.564697  [ 3232/ 6250]
Test Error: 
 Accuracy: 11.9%, Avg loss: 3.764094 

Epoch 8
-------------------------------
loss: 3.521296  [   32/ 6250]
loss: 3.519873  [ 3232